In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        rescale =1./255    
)

img = load_img('real_car_image1.jpg',target_size=(700,700))  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='car', save_format='jpeg'):
    i += 1
    if i > 100:
        break 

In [12]:
import torch
import torchvision
import torchvision.transforms as transforms
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

# Load pre-trained Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# COCO category names
COCO_CATEGORIES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book', 'clock',
    'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

CAR_LABEL = 'car'
CAR_CATEGORY_INDEX = COCO_CATEGORIES.index(CAR_LABEL)

image_dir ='preview'
output_annotated_dir='annotations'
output_annotated_image_dir='annotated images'

device='cpu'

def auto_annotate_directory_faster_rcnn_car_only(image_dir, output_annot_dir, output_annotated_image_dir, confidence_threshold=0.5):
    """
    Automatically annotates cars in all images within a directory using Faster R-CNN.

    Args:
        image_dir (str): Path to the directory containing images.
        output_annot_dir (str): Path to the directory to save annotation text files.
        output_annotated_image_dir (str): Path to directory to save annotated images (optional, can be None).
        confidence_threshold (float): Confidence threshold for car detections.
    """

    if not os.path.exists(output_annot_dir):
        os.makedirs(output_annot_dir)
    if output_annotated_image_dir and not os.path.exists(output_annotated_image_dir):
        os.makedirs(output_annotated_image_dir)

    image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]

    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        annotated_image_rgb, annotations = auto_annotate_faster_rcnn_car_only_single_image(image_path, confidence_threshold)

        # Save annotations to a text file (one per image)
        annot_file_name = os.path.splitext(image_file)[0] + ".txt" # Same name as image, but .txt extension
        annot_file_path = os.path.join(output_annot_dir, annot_file_name)
        with open(annot_file_path, 'w') as f:
            for ann in annotations:
                bbox = ann['bbox']
                confidence = ann['confidence']
                line = f"{CAR_LABEL} {confidence:.4f} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n" # Format: label confidence x_min y_min x_max y_max
                f.write(line)

        if output_annotated_image_dir:
            annotated_image_output_path = os.path.join(output_annotated_image_dir, image_file)
            cv2.imwrite(annotated_image_output_path, cv2.cvtColor(annotated_image_rgb, cv2.COLOR_RGB2BGR)) # Save annotated image

def auto_annotate_faster_rcnn_car_only_single_image(image_path, confidence_threshold=0.5):
    """
    Annotates cars in a single image using Faster R-CNN, keeping only the detection with the highest confidence.
    """
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = transforms.ToTensor()(image_rgb).to(device).unsqueeze(0)

    with torch.no_grad():
        prediction = model(image_tensor)

    boxes = prediction[0]['boxes'].cpu()
    labels = prediction[0]['labels'].cpu()
    scores = prediction[0]['scores'].cpu()

    # --------------------  INSERT THE CODE BLOCK BELOW HERE  --------------------
    # REPLACE the original 'annotated_image = image.copy()...' and the original 'for' loop
    annotated_image = image.copy()
    annotations = []
    highest_confidence = -1.0 # Initialize to a very low value
    best_detection = None

    for i, box in enumerate(boxes):
        score = scores[i]
        label_id = labels[i]
        if label_id == CAR_CATEGORY_INDEX and score > confidence_threshold:
            confidence = float(score.numpy())
            if confidence > highest_confidence: # Check if current detection has higher confidence
                highest_confidence = confidence
                best_detection = { # Store the detection with highest confidence so far
                    'bbox': box.numpy().astype(int),
                    'label': CAR_LABEL,
                    'confidence': confidence
                }

    if best_detection: # If a best detection was found (at least one car above threshold)
        box = best_detection['bbox']
        cv2.rectangle(annotated_image, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
        caption = f"{CAR_LABEL} ({best_detection['confidence']:.2f})"
        cv2.putText(annotated_image, caption, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        annotations = [best_detection] # Create annotation list with only the best detection
    # --------------------  END OF INSERTION  --------------------

    annotated_image_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
    return annotated_image_rgb, annotations



# Example usage for a directory of images:
# image_directory = 'path/to/your/image/directory'  # Replace with path to your directory of images
# output_annotation_directory = 'output_annotations' # Directory to save annotation text files
# output_annotated_images_directory = 'output_annotated_images' # Directory to save images with bounding boxes (optional, set to None to skip)

auto_annotate_directory_faster_rcnn_car_only(image_dir, output_annotated_dir, output_annotated_image_dir, confidence_threshold=0.3)

print(f"Car annotations saved to: {output_annotated_dir}")
if output_annotated_image_dir:
    print(f"Annotated images saved to: {output_annotated_image_dir}")

Car annotations saved to: annotations
Annotated images saved to: annotated images


In [ ]:
import os
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import time

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL image to tensor
    transforms.Normalize(   # Normalize with ImageNet stats
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

class CarDetectionDataset(Dataset):
    """Custom dataset for car detection with text annotations."""
    
    def __init__(self, image_dir, annotations_dir, transform=None):
        """
        Args:
            image_dir (str): Directory with images
            annotations_dir (str): Directory with annotation text files
            transform (callable, optional): Optional transform to be applied
        """
        self.image_dir = image_dir
        self.annotations_dir = annotations_dir
        self.transform = transform
        
        # Get list of image files
        self.image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]
        
        # Sort to ensure consistent ordering
        self.image_files.sort()
        
        print(f"Found {len(self.image_files)} images in {image_dir}")
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        # Load image
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")
        print(f"Original image type: {type(image)}")  # Debug: Should be <class 'PIL.Image.Image'>
        
        # Get annotation filename
        annotation_file = os.path.splitext(self.image_files[idx])[0] + ".txt"
        annotation_path = os.path.join(self.annotations_dir, annotation_file)
        
        boxes = []
        labels = []
        
        # Parse annotation file
        if os.path.exists(annotation_path):
            with open(annotation_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 6:
                        class_name = parts[0]
                        confidence = float(parts[1])
                        x_min = int(float(parts[2]))
                        y_min = int(float(parts[3]))
                        x_max = int(float(parts[4]))
                        y_max = int(float(parts[5]))
                        
                        # Add bounding box
                        boxes.append([x_min, y_min, x_max, y_max])
                        # Class label (assuming 'car' is class 1)
                        labels.append(1)
        
        # Convert to tensors
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        image_id = torch.tensor([idx])
        
        # Calculate areas
        if len(boxes) > 0:
            area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        else:
            area = torch.tensor([])
        
        # Suppose all instances are not crowd
        iscrowd = torch.zeros((len(boxes),), dtype=torch.int64)
        
        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': image_id,
            'area': area,
            'iscrowd': iscrowd
        }
        
        # Apply transformations
        if self.transform:
            image = self.transform(image)
            print(f"Transformed image type: {type(image)}")  # Debug: Should be <class 'torch.Tensor'>
            print(f"Transformed image shape: {image.shape}")  # Debug: Should be [C, H, W]
        
        return image, target

# Use a custom collate function to handle batches
def collate_fn(batch):
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    print(f"Collated images type: {type(images[0])}")  # Debug: Should be <class 'torch.Tensor'>
    print(f"Collated images shape: {images[0].shape}")  # Debug: Should be [C, H, W]
    return images, targets

def get_model_instance_segmentation(num_classes):
    """
    Load a pre-trained Faster R-CNN model and replace the classification head 
    with a new one for our number of classes
    """
    # Load a pre-trained model
    model = fasterrcnn_resnet50_fpn()
    
    # Get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # Replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    return model

def train_model(model, data_loader, optimizer, device, num_epochs=10):
    """
    Train the model
    """
    model.to(device)
    model.train()
    print(f"Starting training for {num_epochs} epochs...")
    
    for epoch in range(num_epochs):
        start_time = time.time()
        epoch_loss = 0
        
        for images, targets in data_loader:
            print(f"Batch images type: {type(images[0])}")  # Debug: Should be <class 'torch.Tensor'>
            print(f"Batch images shape: {images[0].shape}")  # Debug: Should be [C, H, W]
            
            # Move images and targets to the correct device
            images = [image.to(device) for image in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            # Forward pass
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            
            # Backward pass
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
            
            epoch_loss += losses.item()
        
        time_taken = time.time() - start_time
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(data_loader):.4f}, Time: {time_taken:.2f}s")
    
    print("Training complete!")
    return model

def process_video(model, video_path, output_dir, confidence_threshold=0.5, device="cpu"):
    """
    Process a video, detect objects, and save frames with detections
    
    Args:
        model: Trained Faster R-CNN model
        video_path: Path to the video file
        output_dir: Directory to save frames with detections
        confidence_threshold: Minimum confidence score to consider a detection
        device: Device to run the model on (cpu or cuda)
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Video info: {frame_width}x{frame_height}, {fps} fps, {total_frames} total frames")
    
    # Set model to evaluation mode
    model.eval()
    
    frame_count = 0
    detected_frames = 0
    
    # Process each frame
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert frame to RGB and then to tensor
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_tensor = transforms.ToTensor()(frame_rgb).unsqueeze(0).to(device)
        
        # Get prediction
        with torch.no_grad():
            prediction = model(frame_tensor)
        
        # Get boxes, labels, and scores
        boxes = prediction[0]['boxes'].cpu().numpy()
        labels = prediction[0]['labels'].cpu().numpy()
        scores = prediction[0]['scores'].cpu().numpy()
        
        # Filter detections based on confidence threshold
        mask = scores >= confidence_threshold
        boxes = boxes[mask]
        labels = labels[mask]
        scores = scores[mask]
        
        # Save frame if any detections
        if len(boxes) > 0:
            detected_frames += 1
            
            # Create output filename
            output_filename = f"frame_{frame_count:06d}.jpg"
            output_path = os.path.join(output_dir, output_filename)
            
            # Draw bounding boxes on frame
            annotated_frame = frame.copy()
            for box, label, score in zip(boxes, labels, scores):
                x1, y1, x2, y2 = box.astype(int)
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                label_text = f"Car: {score:.2f}"
                cv2.putText(annotated_frame, label_text, (x1, y1 - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            # Save annotated frame
            cv2.imwrite(output_path, annotated_frame)
        
        frame_count += 1
        
        # Print progress
        if frame_count % 100 == 0:
            print(f"Processed {frame_count}/{total_frames} frames ({frame_count/total_frames*100:.1f}%)")
            print(f"Detected objects in {detected_frames} frames so far")
    
    cap.release()
    print(f"Video processing complete. Found objects in {detected_frames} out of {frame_count} frames.")

def main():
    # Paths
    image_dir = "preview"                # Directory containing training images
    annotations_dir = "annotations"      # Directory containing annotation text files
    video_path = "car_vidieo_2.mp4"        # Path to the video file to process
    output_dir = "detected_frames"       # Directory to save frames with detections
    model_save_path = "car_detection_model_1.pth"  # Path to save the trained model
    
    # Check if CUDA is available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Training parameters
    num_classes = 2  # Background + Car
    batch_size = 2
    num_epochs = 10
    learning_rate = 0.005
    
    # Create dataset and dataloader
    dataset = CarDetectionDataset(image_dir, annotations_dir, transform=transform)
    
    # Use a custom collate function to handle batches
    def collate_fn(batch):
        return tuple(zip(*batch))
    
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    
    # Get model
    model = get_model_instance_segmentation(num_classes)
    
    # Define optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=learning_rate, momentum=0.9, weight_decay=0.0005)
    
    # Train the model
    model = train_model(model, data_loader, optimizer, device, num_epochs=num_epochs)
    
    # Save the trained model
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")
    
    # Process video
    process_video(model, video_path, output_dir, confidence_threshold=0.5, device=device)

if __name__ == "__main__":
    main()

Using device: cpu
Found 100 images in preview
Starting training for 10 epochs...
Original image type: <class 'PIL.Image.Image'>
Transformed image type: <class 'torch.Tensor'>
Transformed image shape: torch.Size([3, 700, 700])
Original image type: <class 'PIL.Image.Image'>
Transformed image type: <class 'torch.Tensor'>
Transformed image shape: torch.Size([3, 700, 700])
Batch images type: <class 'torch.Tensor'>
Batch images shape: torch.Size([3, 700, 700])
Original image type: <class 'PIL.Image.Image'>
Transformed image type: <class 'torch.Tensor'>
Transformed image shape: torch.Size([3, 700, 700])
Original image type: <class 'PIL.Image.Image'>
Transformed image type: <class 'torch.Tensor'>
Transformed image shape: torch.Size([3, 700, 700])
Batch images type: <class 'torch.Tensor'>
Batch images shape: torch.Size([3, 700, 700])
Original image type: <class 'PIL.Image.Image'>
Transformed image type: <class 'torch.Tensor'>
Transformed image shape: torch.Size([3, 700, 700])
Original image typ

In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import cv2
from sklearn.metrics import (
    precision_recall_curve, 
    average_precision_score, 
    confusion_matrix, 
    f1_score, 
    precision_score, 
    recall_score
)

class EvaluationDataset(Dataset):
    """Custom dataset for model evaluation with ground truth annotations."""
    
    def __init__(self, image_dir, annotations_dir, transform=None):
        self.image_dir = image_dir
        self.annotations_dir = annotations_dir
        self.transform = transform
        
        self.image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]
        self.image_files.sort()
        
        print(f"Found {len(self.image_files)} images for evaluation")
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        image = cv2.imread(img_path)
        
        if image is None:
            raise ValueError(f"Could not read image {img_path}")
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        annotation_file = os.path.splitext(self.image_files[idx])[0] + ".txt"
        annotation_path = os.path.join(self.annotations_dir, annotation_file)
        
        gt_boxes = []
        gt_labels = []
        
        if os.path.exists(annotation_path):
            with open(annotation_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 6:
                        x_min = int(float(parts[2]))
                        y_min = int(float(parts[3]))
                        x_max = int(float(parts[4]))
                        y_max = int(float(parts[5]))
                        
                        gt_boxes.append([x_min, y_min, x_max, y_max])
                        gt_labels.append(1)  # Car class
        
        if self.transform:
            image = self.transform(image)
        
        return image, {
            'boxes': torch.tensor(gt_boxes, dtype=torch.float32),
            'labels': torch.tensor(gt_labels, dtype=torch.int64)
        }

def calculate_iou(box1, box2):
    """Calculate Intersection over Union (IoU) between two bounding boxes"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = box1_area + box2_area - intersection
    
    return intersection / union if union > 0 else 0

def evaluate_model(model, data_loader, device, iou_threshold=0.5, conf_thresholds=[0.3, 0.5, 0.7]):
    """Comprehensive model evaluation"""
    model.eval()
    model.to(device)
    
    evaluation_results = {}
    
    # Evaluate at multiple confidence thresholds
    for conf_threshold in conf_thresholds:
        all_true_positives = []
        all_predictions = []
        all_ground_truth = []
        all_scores = []
        
        with torch.no_grad():
            for images, targets in data_loader:
                images = [img.to(device) for img in images]
                predictions = model(images)
                
                for i, pred in enumerate(predictions):
                    # Filter predictions
                    mask = pred['scores'] >= conf_threshold
                    pred_boxes = pred['boxes'][mask].cpu()
                    pred_scores = pred['scores'][mask].cpu()
                    
                    gt_boxes = targets[i]['boxes']
                    
                    # Compute matches
                    image_preds = []
                    image_gt = []
                    
                    for pred_box, pred_score in zip(pred_boxes, pred_scores):
                        # Check if prediction matches any ground truth box
                        is_match = any(
                            calculate_iou(pred_box, gt_box) >= iou_threshold 
                            for gt_box in gt_boxes
                        )
                        
                        image_preds.append(1 if is_match else 0)
                        image_gt.append(1 if len(gt_boxes) > 0 else 0)
                        all_scores.append(pred_score.item())
                    
                    all_true_positives.extend(image_preds)
                    all_predictions.extend(image_preds)
                    all_ground_truth.extend(image_gt)
        
        # Compute metrics
        precision = precision_score(all_ground_truth, all_predictions, zero_division=0)
        recall = recall_score(all_ground_truth, all_predictions, zero_division=0)
        f1 = f1_score(all_ground_truth, all_predictions, zero_division=0)
        
        precision_curve, recall_curve, _ = precision_recall_curve(all_ground_truth, all_scores)
        avg_precision = average_precision_score(all_ground_truth, all_scores)
        
        evaluation_results[conf_threshold] = {
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'average_precision': avg_precision,
            'precision_curve': precision_curve,
            'recall_curve': recall_curve
        }
    
    return evaluation_results

def plot_results(evaluation_results):
    """Create comprehensive visualization of evaluation results"""
    plt.figure(figsize=(20, 15))
    
    # 1. Precision-Recall Curves
    plt.subplot(2, 3, 1)
    for threshold, results in evaluation_results.items():
        plt.plot(
            results['recall_curve'], 
            results['precision_curve'], 
            label=f'Threshold {threshold} (AP={results["average_precision"]:.2f})'
        )
    plt.title('Precision-Recall Curves')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.legend()
    
    # 2. Bar Plot of Metrics
    plt.subplot(2, 3, 2)
    thresholds = list(evaluation_results.keys())
    precisions = [results['precision'] for results in evaluation_results.values()]
    recalls = [results['recall'] for results in evaluation_results.values()]
    f1_scores = [results['f1_score'] for results in evaluation_results.values()]
    
    x = np.arange(len(thresholds))
    width = 0.25
    plt.bar(x - width, precisions, width, label='Precision')
    plt.bar(x, recalls, width, label='Recall')
    plt.bar(x + width, f1_scores, width, label='F1 Score')
    plt.title('Metrics at Different Thresholds')
    plt.xlabel('Confidence Threshold')
    plt.ylabel('Score')
    plt.xticks(x, thresholds)
    plt.legend()
    
    # 3. Heatmap of IoU Distribution
    plt.subplot(2, 3, 3)
    iou_values = np.random.rand(10, 10)  # Placeholder, replace with actual IoU calculations
    sns.heatmap(iou_values, cmap='YlGnBu', annot=True, fmt='.2f')
    plt.title('IoU Distribution')
    
    # 4. Performance Radar Chart
    plt.subplot(2, 3, 4, polar=True)
    metrics = [
        evaluation_results[0.5]['precision'], 
        evaluation_results[0.5]['recall'], 
        evaluation_results[0.5]['f1_score'],
        evaluation_results[0.5]['average_precision']
    ]
    angles = np.linspace(0, 2*np.pi, len(metrics), endpoint=False)
    metrics += metrics[:1]
    angles = np.concatenate((angles, [angles[0]]))
    
    plt.polar(angles, metrics, 'o-', linewidth=2)
    plt.fill(angles, metrics, alpha=0.25)
    plt.title('Performance Radar Chart')
    plt.xticks(angles[:-1], ['Precision', 'Recall', 'F1', 'AP'])
    
    # 5. Average Precision Comparison
    plt.subplot(2, 3, 5)
    aps = [results['average_precision'] for results in evaluation_results.values()]
    plt.bar(thresholds, aps)
    plt.title('Average Precision')
    plt.xlabel('Confidence Threshold')
    plt.ylabel('Average Precision')
    
    plt.tight_layout()
    plt.savefig('model_evaluation_results.png', dpi=300)

def main():
    # Paths
    image_dir = "preview"
    annotations_dir = "annotations"
    model_path = "car_detection_model_1.pth"
    
    # Check if model file exists
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file {model_path} not found!")
    
    # Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Evaluation device: {device}")
    
    # Transformations
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225]
        )
    ])
    
    # Create dataset and dataloader
    dataset = EvaluationDataset(image_dir, annotations_dir, transform=transform)
    data_loader = DataLoader(dataset, batch_size=2, shuffle=False, 
                              collate_fn=lambda x: tuple(zip(*x)))
    
    # Load model
    num_classes = 2  # Background + Car
    model = fasterrcnn_resnet50_fpn(weights=None)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    # Load trained weights
    model.load_state_dict(torch.load(model_path, map_location=device))
    
    # Evaluate model
    evaluation_results = evaluate_model(model, data_loader, device)
    
    # Plot comprehensive results
    plot_results(evaluation_results)
    
    # Print summary
    print("\nEvaluation Summary:")
    for threshold, metrics in evaluation_results.items():
        print(f"\nConfidence Threshold: {threshold}")
        print(f"Precision: {metrics['precision']:.4f}")
        print(f"Recall: {metrics['recall']:.4f}")
        print(f"F1 Score: {metrics['f1_score']:.4f}")
        print(f"Average Precision: {metrics['average_precision']:.4f}")

if __name__ == "__main__":
    main()